In [0]:
from pandas import DataFrame, Series
import pandas as pd
from dateutil.parser import parse
import datetime as datetime
import matplotlib.pyplot as plt

In [0]:
#load the dataset
def load_data(data_location, region_location):
  data = pd.read_csv(data_location)
  region_info = pd.read_csv(region_location)
  region_info.drop_duplicates('LocationID','first',inplace=True)
  return data, region_info

In [0]:
data, region_info = load_data('/content/drive/capstone dataset/yellow_tripdata_2018-08.csv','/content/drive/capstone dataset/taxizones.csv')

In [0]:
data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-08-01 00:44:35,2018-08-01 01:03:22,1,5.60,1,N,238,79,1,19.0,0.5,0.5,4.00,0.0,0.3,24.30
1,1,2018-08-01 00:02:19,2018-08-01 00:02:31,1,0.00,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80
2,1,2018-08-01 00:13:25,2018-08-01 00:24:40,1,2.90,1,N,138,7,1,11.5,0.5,0.5,2.55,0.0,0.3,15.35
3,1,2018-08-01 00:10:37,2018-08-01 00:49:10,1,8.40,1,N,231,7,1,30.0,0.5,0.5,7.80,0.0,0.3,39.10
4,1,2018-08-01 00:02:18,2018-08-01 00:07:32,2,0.70,1,N,79,148,1,5.5,0.5,0.5,0.00,0.0,0.3,6.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7849129,1,2018-08-31 23:54:46,2018-09-01 00:04:36,4,1.30,1,N,148,79,2,7.5,0.5,0.5,0.00,0.0,0.3,8.80
7849130,2,2018-08-31 23:24:09,2018-08-31 23:32:33,1,1.37,1,N,113,231,2,7.5,0.5,0.5,0.00,0.0,0.3,8.80
7849131,2,2018-08-31 23:35:09,2018-08-31 23:47:18,1,1.27,1,N,231,148,1,9.0,0.5,0.5,5.08,0.0,0.3,15.38
7849132,2,2018-08-31 23:55:23,2018-09-01 00:06:14,1,2.08,1,N,114,87,1,9.5,0.5,0.5,1.08,0.0,0.3,11.88


In [0]:
del data['RatecodeID']
del data['store_and_fwd_flag']
del data['payment_type']
del data['passenger_count']
del data['VendorID']

In [0]:
def del_negative_values(filtered_data):
  #[fare_amount]
  fare_index = filtered_data[filtered_data['fare_amount'] < 2.5].index
  filtered_data = filtered_data.drop(fare_index)
  #[extra]
  fare_index_extra = filtered_data[filtered_data['extra'] < 0].index
  filtered_data = filtered_data.drop(fare_index_extra)
  #[tolls_amount]
  fare_index_tolls = filtered_data[filtered_data['tolls_amount'] < 0].index
  filtered_data = filtered_data.drop(fare_index_tolls)
  #[tip_amount]
  fare_index_tip = filtered_data[filtered_data['tip_amount'] < 0].index
  filtered_data = filtered_data.drop(fare_index_tip)
  #[improvement_surcharge]
  filtered_data_improvement = filtered_data[filtered_data['improvement_surcharge'] != 0.3].index
  filtered_data = filtered_data.drop(filtered_data_improvement)
  #[mta_tax]
  filtered_data_mtatax =  filtered_data[(filtered_data['mta_tax'] != 0.5) & (filtered_data['mta_tax'] != 0)].index
  filtered_data = filtered_data.drop(filtered_data_mtatax)
  return filtered_data

In [0]:
# datasize 8713831 -> 8703215
newData = del_negative_values(data)

In [0]:
# format of the date '2018-06-01 00:00:00'
def date_check(result,start_date, end_date, attribute):
  result = result.sort_values(by=attribute, ascending= True)
  result.reset_index(inplace = True, drop=True)
  pickup_start = result[result[attribute] >= start_date].index[0]
  pickup_end = result[result[attribute] >= end_date].index[0]
  newData = result.iloc[pickup_start:pickup_end, :]
  return newData

In [0]:
newData = date_check(newData,'2018-08-01 00:00:00', '2018-09-01 00:00:00','tpep_pickup_datetime')
newData = date_check(newData,'2018-08-01 00:00:00', '2018-09-01 00:00:00','tpep_dropoff_datetime')

In [0]:
def fare_abnormal(newData):
  #[tolls_amount]
  tolls_amount = newData[(newData['tolls_amount'] > 30)].index
  newData = newData.drop(tolls_amount)
  #[trip_distance]
  trip_dis = newData[(newData['trip_distance'] > 50)|(newData['trip_distance'] < 0.5)].index
  newData = newData.drop(trip_dis)
  #[fare_amount]
  fare_amount = newData[newData['fare_amount'] > 75].index
  newData = newData.drop(fare_amount)
  return newData

In [0]:
newData = fare_abnormal(newData)

In [0]:
#new column duration
newData['duration'] = (pd.to_datetime(newData['tpep_dropoff_datetime']) - pd.to_datetime(newData['tpep_pickup_datetime']))
newData['duration'] = newData['duration'].map(lambda x:x.total_seconds()/60)
#[duration]
duration = newData[(newData['duration']>180)|(newData['duration']<5)].index
newData = newData.drop(duration)
#new column average speed - km/h
newData['avg_speed'] = newData['trip_distance']/newData['duration']*60
#[avg_speed]
avg_speed = newData[(newData['avg_speed'] > 129)|(newData['avg_speed'] < 5)].index
newData = newData.drop(avg_speed)

In [0]:
newData

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration,avg_speed
48,2018-08-01 00:00:17,2018-08-01 00:05:18,1.33,249,186,6.0,0.5,0.5,1.46,0.00,0.3,8.76,5.016667,15.906977
50,2018-08-01 00:00:02,2018-08-01 00:05:26,1.10,142,237,6.0,0.5,0.5,1.45,0.00,0.3,8.75,5.400000,12.222222
52,2018-08-01 00:00:16,2018-08-01 00:05:30,1.67,239,166,7.0,0.5,0.5,1.66,0.00,0.3,9.96,5.233333,19.146497
54,2018-08-01 00:00:11,2018-08-01 00:05:34,1.08,249,231,6.0,0.5,0.5,0.00,0.00,0.3,7.30,5.383333,12.037152
63,2018-08-01 00:00:22,2018-08-01 00:05:55,1.70,144,170,7.0,0.5,0.5,1.65,0.00,0.3,9.95,5.550000,18.378378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7835519,2018-08-31 23:31:13,2018-08-31 23:59:57,18.40,132,75,52.0,0.0,0.5,0.00,5.76,0.3,58.56,28.733333,38.422274
7835520,2018-08-31 23:48:01,2018-08-31 23:59:57,1.29,230,246,9.0,0.5,0.5,3.09,0.00,0.3,13.39,11.933333,6.486034
7835521,2018-08-31 23:45:40,2018-08-31 23:59:57,3.60,230,74,13.5,0.5,0.5,4.00,0.00,0.3,18.80,14.283333,15.122520
7835522,2018-08-31 23:52:48,2018-08-31 23:59:58,1.50,236,141,7.5,0.5,0.5,1.75,0.00,0.3,10.55,7.166667,12.558140


In [0]:
newData.to_csv('/content/drive/capstone dataset/cleaned_2018_08.csv', index = False)

In [0]:
final = pd.read_csv('/content/drive/capstone dataset/cleaned_2018_07.csv')

In [0]:
final

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration,avg_speed
0,2018-07-01 00:00:16,2018-07-01 00:05:21,1.10,113,170,6.0,0.5,0.5,1.50,0.0,0.3,8.80,5.083333,12.983607
1,2018-07-01 00:00:08,2018-07-01 00:05:30,0.70,230,162,5.5,0.5,0.5,0.00,0.0,0.3,6.80,5.366667,7.826087
2,2018-07-01 00:00:47,2018-07-01 00:05:57,0.87,170,233,5.5,0.5,0.5,1.36,0.0,0.3,8.16,5.166667,10.103226
3,2018-07-01 00:00:43,2018-07-01 00:06:04,1.80,48,68,7.0,0.5,0.5,1.65,0.0,0.3,9.95,5.350000,20.186916
4,2018-07-01 00:00:50,2018-07-01 00:06:11,1.53,239,151,6.5,0.5,0.5,1.56,0.0,0.3,9.36,5.350000,17.158879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6046503,2018-07-31 23:54:30,2018-07-31 23:59:57,1.24,166,238,6.5,0.5,0.5,1.00,0.0,0.3,8.80,5.450000,13.651376
6046504,2018-07-31 23:44:19,2018-07-31 23:59:57,3.10,230,144,13.5,0.5,0.5,0.00,0.0,0.3,14.80,15.633333,11.897655
6046505,2018-07-31 23:39:27,2018-07-31 23:59:58,5.80,137,260,20.0,0.5,0.5,2.00,0.0,0.3,23.30,20.516667,16.961820
6046506,2018-07-31 23:45:06,2018-07-31 23:59:59,4.00,164,75,14.0,0.5,0.5,5.00,0.0,0.3,20.30,14.883333,16.125420


In [0]:
 'tpep_pickup_datetime'
 'tpep_dropoff_datetime'
 'trip_distance'
 'PULocationID' ---> pickup locationID(1-260)
 'DOLocationID' ---> dropoff locationID(1-260)
 'fare_amount' ---> meter showed payment
 'total_amount' ---> fare_amount + different kinds of surcharge
 'Duration' ---> minutes
 'avg_speed' ---> average speed 
 'tip_amount' ---> tips
 'tolls_amount' ---> tolls
 'improvement_surcharge' ---> surcharge
 'extra'
 'mta_tax'